In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [2]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>',
                                          eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

In [3]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-'*25)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
-------------------------
</s>
<usr>
<pad>
<sys>


In [4]:
import pandas as pd
import tqdm
import urllib.request

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
print('챗 봇 데 이 터 의 개 수 :', len(train_data))

챗 봇 데 이 터 의 개 수 : 11823


In [6]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)

  yield bos_token + sent + eos_token
  # yield는 값을 하나씩“잠깐 내보내고”함수 실행을 멈춘 뒤, 다음 호출 때 다시 이어서 실행할 수 있게 해줘요.


In [7]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

# 배치 크기 만큼 데이터를 구성하되 패딩 토큰으로 패딩을 진행.
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,),
                               padding_values=tokenizer.pad_token_id)

In [8]:
# 첫 배치 출력
for batch in dataset:
  print(batch)
  break

tf.Tensor(
[[    1     2 45231  7788 10715  8705  6969  7661     4 25030  7799 43558
  11242  9111  6962 13158 25856     1]], shape=(1, 18), dtype=int32)


In [9]:
# 첫 번 째 배 치(32개 의 샘 플) 중 첫 번 째 샘 플 출 력
print(tokenizer.decode(batch[0]))

</s><usr> 힘들어서 결혼할까봐<sys> 도피성 결혼은 하지 않길 바라요.</s>


In [10]:
# 옵티마이저 결정
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# 전체 데이터의 개수를 배치 크기로 나누면 하나의 에포크에서 실행되는 학습 횟수가 계산됨
steps = len(train_data) // batch_size + 1

EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
    with tf.GradientTape() as tape:
      result = model(batch, labels=batch)
      loss = result[0]
      batch_loss = tf.reduce_mean(loss)

    grads = tape.gradient(batch_loss, model.trainable_variables)
    adam.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss += batch_loss / steps
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 0.0201111156


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 0.0122005166


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    3] cost = 0.00822486915


In [11]:
text = '오 늘 도 좋 은 하 루!'
sent = '<usr>' + text + '<sys>'
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print('정 수 인 코 딩 후 :', input_ids)
print('정 수 인 코 딩 을 재 복 원 :', tokenizer.decode(input_ids[0]))

정 수 인 코 딩 후 : tf.Tensor([[    1     2  9114 10061  9095  9677  9674  9078  9930   376     4]], shape=(1, 11), dtype=int32)
정 수 인 코 딩 을 재 복 원 : </s><usr> 오 늘 도 좋 은 하 루!<sys>
